In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 52.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=33b5eaecc5d950f5bd51ccb50c1a6720674287510129370b6254b489bc90e260
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

if __name__ == "__main__":
  spark = SparkSession \
        .builder \
        .appName("exploring_spark") \
        .getOrCreate()

anime = spark.read.csv("anime.csv", header = True).cache()
anime.printSchema()
anime.show()


root
 |-- anime_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- members: string (nullable = true)

+--------+--------------------+--------------------+-----+--------+------+-------+
|anime_id|                name|               genre| type|episodes|rating|members|
+--------+--------------------+--------------------+-----+--------+------+-------+
|   32281|      Kimi no Na wa.|Drama, Romance, S...|Movie|       1|  9.37| 200630|
|    5114|Fullmetal Alchemi...|Action, Adventure...|   TV|      64|  9.26| 793665|
|   28977|            Gintama°|Action, Comedy, H...|   TV|      51|  9.25| 114262|
|    9253|         Steins;Gate|    Sci-Fi, Thriller|   TV|      24|  9.17| 673572|
|    9969|       Gintama&#039;|Action, Comedy, H...|   TV|      51|  9.16| 151266|
|   32935|Haikyuu!!: Karasu...|Comedy, Drama, Sc...|   TV|      

In [ ]:
# we're able to extract data via sql queries..
anime.createOrReplaceTempView("ratings")
ratings_df = spark.sql("SELECT * from ratings")
ratings_df.show()

+--------+--------------------+--------------------+-----+--------+------+-------+
|anime_id|                name|               genre| type|episodes|rating|members|
+--------+--------------------+--------------------+-----+--------+------+-------+
|   32281|      Kimi no Na wa.|Drama, Romance, S...|Movie|       1|  9.37| 200630|
|    5114|Fullmetal Alchemi...|Action, Adventure...|   TV|      64|  9.26| 793665|
|   28977|            Gintama°|Action, Comedy, H...|   TV|      51|  9.25| 114262|
|    9253|         Steins;Gate|    Sci-Fi, Thriller|   TV|      24|  9.17| 673572|
|    9969|       Gintama&#039;|Action, Comedy, H...|   TV|      51|  9.16| 151266|
|   32935|Haikyuu!!: Karasu...|Comedy, Drama, Sc...|   TV|      10|  9.15|  93351|
|   11061|Hunter x Hunter (...|Action, Adventure...|   TV|     148|  9.13| 425855|
|     820|Ginga Eiyuu Densetsu|Drama, Military, ...|  OVA|     110|  9.11|  80679|
|   15335|Gintama Movie: Ka...|Action, Comedy, H...|Movie|       1|  9.10|  72534|
|   

In [ ]:
type_df = ratings_df.groupBy("type").count()
type_df.show()

+-------+-----+
|   type|count|
+-------+-----+
|     TV| 3787|
|   null|   25|
|Special| 1676|
|    OVA| 3311|
|  Music|  488|
|  Movie| 2348|
|    ONA|  659|
+-------+-----+



In [ ]:
# transforming..
length_df = spark.sql("SELECT * from ratings WHERE ratings.episodes > 50 AND 150 > ratings.episodes")
length_df.show()

+--------+--------------------+--------------------+----+--------+------+-------+
|anime_id|                name|               genre|type|episodes|rating|members|
+--------+--------------------+--------------------+----+--------+------+-------+
|    5114|Fullmetal Alchemi...|Action, Adventure...|  TV|      64|  9.26| 793665|
|   28977|            Gintama°|Action, Comedy, H...|  TV|      51|  9.25| 114262|
|    9969|       Gintama&#039;|Action, Comedy, H...|  TV|      51|  9.16| 151266|
|   11061|Hunter x Hunter (...|Action, Adventure...|  TV|     148|  9.13| 425855|
|     820|Ginga Eiyuu Densetsu|Drama, Military, ...| OVA|     110|  9.11|  80679|
|     263|      Hajime no Ippo|Comedy, Drama, Sh...|  TV|      75|  8.83| 157670|
|      19|             Monster|Drama, Horror, My...|  TV|      74|  8.72| 247562|
|   12431|       Uchuu Kyoudai|Comedy, Sci-Fi, S...|  TV|      99|  8.59|  72958|
|     170|           Slam Dunk|Comedy, Drama, Sc...|  TV|     101|  8.56|  82570|
|   30230|Diamon

In [ ]:
selected_df = spark.sql("SELECT * from ratings WHERE rating > 8.0 AND ratings.type == 'TV'").orderBy("rating", ascending =False)
selected_df.show()

+--------+--------------------+--------------------+----+--------+------+-------+
|anime_id|                name|               genre|type|episodes|rating|members|
+--------+--------------------+--------------------+----+--------+------+-------+
|   30120|Spoon-hime no Swi...|     Adventure, Kids|  TV| Unknown|  9.60|     47|
|    5114|Fullmetal Alchemi...|Action, Adventure...|  TV|      64|  9.26| 793665|
|   28977|            Gintama°|Action, Comedy, H...|  TV|      51|  9.25| 114262|
|    9253|         Steins;Gate|    Sci-Fi, Thriller|  TV|      24|  9.17| 673572|
|    9969|       Gintama&#039;|Action, Comedy, H...|  TV|      51|  9.16| 151266|
|   32935|Haikyuu!!: Karasu...|Comedy, Drama, Sc...|  TV|      10|  9.15|  93351|
|   11061|Hunter x Hunter (...|Action, Adventure...|  TV|     148|  9.13| 425855|
|   15417|Gintama&#039;: En...|Action, Comedy, H...|  TV|      13|  9.11|  81109|
|    4181|Clannad: After Story|Drama, Fantasy, R...|  TV|      24|  9.06| 456749|
|     918|      

In [ ]:
# now loading..
selected_df.write.json("filtered.json")

In [ ]:
#next: maybe add a database configuration